# Model performance

Comparing baseline vs current model performance, and investigation into areas for improvement.

In [1]:
import itertools
import json
import random
import yaml

import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
# Move project root folder
%cd ..

/Users/mitchell/projects/personal/f1-rating-system


/Users/mitchell/Library/Caches/pypoetry/virtualenvs/f1-rating-system-bj5GO79q-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
with open("params.yaml") as conf_file:
    CONFIG = yaml.safe_load(conf_file)

# Baseline model: Random expectation

In [4]:
features_df = pd.read_csv(CONFIG["data"]["features_path"])
features_df["prevRaces"] = features_df.groupby("driverId").cumcount()

In [5]:
yr_df = features_df[["year", "round"]].drop_duplicates()
score_list = []

for i in range(5):
    error_list = []

    for _, (yr, rnd) in yr_df.iterrows():
        valid_ix = (features_df["year"] == yr) & (features_df["round"] == rnd)
        ran_results = list(features_df.loc[valid_ix, "driverId"])
        random.shuffle(ran_results)

        for ix_1, ix_2 in itertools.combinations(features_df[valid_ix].index, 2):
            dri_i = features_df.loc[ix_1, "driverId"]
            pos_i = features_df.loc[ix_1, "mapPosition"]
            dri_j = features_df.loc[ix_2, "driverId"]
            pos_j = features_df.loc[ix_2, "mapPosition"]

            # continue if drivers in same car
            if pos_i == pos_j:
                continue
            
            # get expected result - random shuffle
            if ran_results.index(dri_i) < ran_results.index(dri_j):
                e_i = 1
                e_j = 0
            else:
                e_i = 0
                e_j = 1

            # get actual result 
            if pos_i < pos_j:
                a_i = 1
                a_j = 0
            else:
                a_i = 0
                a_j = 1

            error_list.append({
                "pred": e_i,
                "true": a_i
            })

    err_df = pd.DataFrame(error_list)
    root_brier_score = np.sqrt(metrics.brier_score_loss(err_df["true"], err_df["pred"]))
    score_list.append(root_brier_score)

print(f"Baseline model performance: {round(np.mean(score_list), 4)}")

Baseline model performance: 0.7081


# Current model: Multiplayer and assymetric model

In [7]:
with open("models/metrics.json") as infile:
    perf_dict = json.load(infile)

print(f"Current model performance : {round(perf_dict['RMSE'], 4)}")

Current model performance : 0.4585


# Error EDA

TBC

In [8]:
# sn.displot(err_df.sample(20000), x="elo_diff", y="squared_error", kind="kde")